In [14]:
from time import perf_counter
import numpy as np,os
from imblearn.under_sampling import InstanceHardnessThreshold, EditedNearestNeighbours
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data

In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [6]:
#ENN already done in Noisy.csv, but kind_sel='mode' here, so here aggressive than before
imbs = {'ENN':EditedNearestNeighbours(n_neighbors=5,kind_sel='mode'), 
       'SmoteEnc':SMOTEENN(),'IHThreshold':InstanceHardnessThreshold()}
models = {}
for im,samp in imbs.items():
    for c,clf in CLFS.items():
        models[(im,c)] = Pipeline([('samp',samp),('clf',clf)])
    
models.keys(),len(models)

(dict_keys([('ENN', 'dt'), ('ENN', 'lr'), ('ENN', 'nb'), ('ENN', 'svm'), ('ENN', 'knn'), ('ENN', 'rf'), ('SmoteEnc', 'dt'), ('SmoteEnc', 'lr'), ('SmoteEnc', 'nb'), ('SmoteEnc', 'svm'), ('SmoteEnc', 'knn'), ('SmoteEnc', 'rf'), ('IHThreshold', 'dt'), ('IHThreshold', 'lr'), ('IHThreshold', 'nb'), ('IHThreshold', 'svm'), ('IHThreshold', 'knn'), ('IHThreshold', 'rf')]),
 18)

In [20]:
path = "Builtin32.csv"
cols = pd.MultiIndex.from_product([imbs.keys(),CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
df = pd.read_csv("Builtin3.csv",header=[0,1,2],index_col=0)

In [22]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        if df.loc[d,(k[0],k[1],slice(None))].isna().sum()==0:
            continue
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = np.nanmean(r[f])
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv("Builtin32.csv")

0
activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)

1
groovy-1_6_BETA_1.csv noise:0.128, imb:6.017,117,704, Shape:(821, 65)

2
activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)

3
wicket-1.3.0-incubating-beta-1.csv noise:0.164, imb:4.806,288,1384, Shape:(1672, 65)

4
jruby-1.1.csv noise:0.175, imb:3.540,161,570, Shape:(731, 65)

5
jruby-1.4.0.csv noise:0.190, imb:3.890,200,778, Shape:(978, 65)

6
lucene-2.3.0.csv noise:0.204, imb:4.031,160,645, Shape:(805, 65)

7
hbase-0.95.2.csv noise:0.260, imb:15.088,114,1720, Shape:(1834, 65)

8
lucene-3.0.0.csv noise:0.185, imb:6.037,190,1147, Shape:(1337, 65)

9
camel-2.9.0.csv noise:0.044, imb:34.600,200,6920, Shape:(7120, 65)

10
wicket-1.5.3.csv noise:0.064, imb:26.720,93,2485, Shape:(2578, 65)

11
lucene-3.1.csv noise:0.120, imb:7.477,331,2475, Shape:(2806, 65)

12
groovy-1_6_BETA_2.csv noise:0.096, imb:7.583,103,781, Shape:(884, 65)

13
activemq-5.2.0.csv noise:0.113, imb:12.247,154,1886, Shape:(2040,

0

In [13]:
np.nanmean(p)

3.5